In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
import numpy as np 
import scipy
import pandas as pd
from matplotlib import pyplot as plt 
import seaborn as sns
import os # imports for loading data
from tqdm import tqdm

In [ ]:
from prophet import Prophet
from prophet.plot import plot_plotly
import matplotlib.pyplot as plt
import plotly.offline as py
py.init_notebook_mode()
%matplotlib inline
from neuralprophet import NeuralProphet

In [ ]:
# Importing the signal in
df = pd.read_csv(os.path.abspath('turn_over.csv'), index_col=False) 
print(f"Successfully read in {df.size} values.")

print("First 5 values:")
df.head(5)

In [ ]:
list_Month = [i.split(" ")[0][0:2] for i in  list(df['CALENDAR_DATE'])]
list_Date = [i.split(" ")[0][6:8] for i in  list(df['CALENDAR_DATE'])]
list_Year = [i.split(" ")[0][3:5] for i in  list(df['CALENDAR_DATE'])]

df['Month'] = list_Month
df['Date'] = list_Date
df['Year'] = list_Year

In [ ]:
df['ds'] = pd.DatetimeIndex(df['Year']+'-'+df['Month']+'-'+df['Date'])

In [ ]:
df= df.rename(columns=str.lower)

In [ ]:
df = df.drop_duplicates(inplace=False,keep='first')

In [ ]:
df.reset_index(inplace=True,drop=True)

In [ ]:
df = df.groupby(['ds','cust_id','jurisdiction'])['turnover'].sum()
df = pd.DataFrame(df)
df = df.rename_axis(['ds', 'cust_id', 'jurisdiction']).reset_index()

In [ ]:
df['ds'].agg(['min', 'max'])
#334 days`

In [ ]:
df.cust_id.nunique()

In [ ]:
juris = list(set(df['jurisdiction']))
juris

In [ ]:
df1 = df[df['jurisdiction']=='DK'].reset_index(drop=True)
df2 = df[df['jurisdiction']=='BE'].reset_index(drop=True)
df3 = df[df['jurisdiction']=='MT'].reset_index(drop=True)
df4 = df[df['jurisdiction']=='IE'].reset_index(drop=True)
df5 = df[df['jurisdiction']=='DE'].reset_index(drop=True)
df6 = df[df['jurisdiction']=='NT'].reset_index(drop=True)
df7 = df[df['jurisdiction']=='SE'].reset_index(drop=True)
df8 = df[df['jurisdiction']=='EE'].reset_index(drop=True)
df9 = df[df['jurisdiction']=='RO'].reset_index(drop=True)
df10 = df[df['jurisdiction']=='IT'].reset_index(drop=True)
df11 = df[df['jurisdiction']=='UK'].reset_index(drop=True)


In [ ]:
df

In [ ]:
df['turnover'].sum()

In [ ]:
df1.head()

In [ ]:
df1.shape

In [ ]:
df1.turnover.sum()

In [ ]:
df1.cust_id.nunique()

In [ ]:
594*334

In [ ]:
def addzeros(dataframe):
    juris = dataframe['jurisdiction'][0]
    my_dates = pd.date_range('2021-07-01', '2022-05-30', freq='D')
    idx= pd.MultiIndex.from_product([my_dates, dataframe.cust_id.unique()],names=['ds', 'cust_id'])
    dataframe = dataframe.set_index(['ds', 'cust_id']).reindex(idx).fillna(0).reset_index()
    dataframe['jurisdiction'] = juris
    return (dataframe)

In [ ]:
df1 = addzeros(df1)
df2 = addzeros(df2)
df3 = addzeros(df3)
df4 = addzeros(df4)
df5 = addzeros(df5)
df6 = addzeros(df6)
df7 = addzeros(df7)
df8 = addzeros(df8)
df9 = addzeros(df9)
df10 = addzeros(df10)
df11 = addzeros(df11)

In [ ]:
dataframe = pd.concat([df1, df2,df3,df4,df5,df6,df7,df8,df9,df10,df11], ignore_index=True)

In [ ]:
dataframe

In [ ]:
dataframe.groupby('cust_id')['turnover'].sum().sort_values(ascending=False)

# Prophet

In [ ]:
#implementing it here for 2 customers to report the values in the thesis
#5520
customer = dataframe[dataframe["cust_id"] == 5520].reset_index()
#10081
#customer = dataframe[dataframe["cust_id"] == 10081].reset_index()


In [ ]:
customer = customer.drop(['index','cust_id','jurisdiction'],axis=1)

In [ ]:
customer.head()

In [ ]:
#customer = customer.drop(['index','cust_id','jurisdiction'],axis=1)

customer['square_root'] = np.sqrt(customer['turnover'])
customer['square_root'] = customer['square_root'].dropna()

forecast_train = customer['square_root'].values[:268]
forecast_test = customer['square_root'].values[268:]

# extract the times
forecast_train_times = customer['square_root'].index[:268]
forecast_test_times = customer['square_root'].index[268:]


In [ ]:
k = customer.drop(['turnover'],axis = 1)

df_train = k[:268]
df_test = k[268:]

from statsmodels.tsa.seasonal import seasonal_decompose

decomposition_obj = seasonal_decompose(
    x=df_train.set_index('ds'), 
    model='additive')

In [ ]:
fig, ax = plt.subplots(4, 1, figsize=(12, 12))

# Observed time series.
decomposition_obj.observed.plot(ax=ax[0])
ax[0].set(title='observed')


# Trend component. 
decomposition_obj.trend.plot(label='fit', ax=ax[1])
#df[mask][['ds', 'trend']].set_index('ds').plot(c=sns_c[1], ax=ax[1])
ax[1].legend(loc='lower right')
ax[1].set(title='trend')

# Seasonal component. 
decomposition_obj.seasonal.plot(label='fit', ax=ax[2])
# df.assign(seasonal = lambda x: x['yearly_seas'] + x['monthly_seas'] + x['end_of_year']) \
#     [mask][['ds', 'seasonal']] \
#     .set_index('ds')\
#     .plot(c=sns_c[2], ax=ax[2])
ax[2].legend(loc='lower right')
ax[2].set(title='seasonal')

# Residual.
decomposition_obj.resid.plot(label='fit', ax=ax[3])
#df[mask][['ds', 'noise']].set_index('ds').plot(c=sns_c[3], ax=ax[3])
ax[3].legend(loc='lower right')
ax[3].set(title='residual')

fig.suptitle('Time Series Decomposition', y=1.01)
plt.tight_layout()

In [ ]:
customer.shape

In [ ]:
    c = customer.drop(['square_root'], axis=1)
    c.columns = ['ds', 'y']
    
    threshold_date = pd.to_datetime('2022-04-01')
    mask = c['ds'] < threshold_date
    # Split the data and select `ds` and `y` columns.
    df_train = c[mask][['ds', 'y']]
    df_test = c[~ mask][['ds', 'y']]
    
    
    m = Prophet(interval_width=0.95,daily_seasonality=True , seasonality_mode = 'multiplicative'
                ,weekly_seasonality = True, mcmc_samples = 100)
    m.add_seasonality(name='monthly', period=30.5, fourier_order=5)
    
    
    model = m.fit(df_train)
    future = m.make_future_dataframe(periods=30,freq='D')
    forecast = m.predict(future)

In [ ]:
mask2 = forecast['ds'] < threshold_date

forecast_train = forecast[mask2]
forecast_test = forecast[~ mask2]

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_true=df_train['y'], y_pred=forecast_train['yhat'],squared = False)


In [ ]:
import seaborn as sns
sns.set_style('darkgrid', {'axes.facecolor': '.9'})
sns.set_palette(palette='deep')
sns_c = sns.color_palette(palette='deep')
%matplotlib inline

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [ ]:
fig, ax = plt.subplots()
sns.set(rc={'figure.figsize':(20,8.27)})
ax.fill_between(
    x=forecast['ds'],
    y1=forecast['yhat_lower'],
    y2=forecast['yhat_upper'],
    color=sns_c[2], 
    alpha=0.25,
    label=r'0.95 credible_interval'
)

sns.lineplot(x='ds', y='y', label='y_train', data=df_train, ax=ax)
sns.lineplot(x='ds', y='y', label='y_test', data=df_test, ax=ax)
sns.lineplot(x='ds', y='yhat', label='y_hat', data=forecast, ax=ax)
ax.axvline(threshold_date, color=sns_c[3], linestyle='--', label='train test split')
ax.legend(loc='upper left')
ax.set(title='Dependent Variable', ylabel='');

In [ ]:
#ZOOM

fig, ax = plt.subplots()

ax.fill_between(
    x=forecast_test['ds'],
    y1=forecast_test['yhat_lower'],
    y2=forecast_test['yhat_upper'],
    color=sns_c[2], 
    alpha=0.25,
    label=r'0.95 credible_interval'
)

sns.lineplot(x='ds', y='y', label='y_test', data=df_test, ax=ax)
sns.lineplot(x='ds', y='yhat', label='y_hat', data=forecast_test, ax=ax)
ax.legend(loc='lower right')
ax.set(title='Dependent Variable', ylabel='');

In [ ]:
m.plot(forecast)

In [ ]:
m.plot_components(forecast)

# implementing the model without completing the dataset

In [ ]:
customer = df[df["cust_id"] == 5520].reset_index()

In [ ]:
    #c = customer.drop(['index','cust_id','jurisdiction'] ,axis=1)
    c.columns = ['ds', 'y']
    
    threshold_date = pd.to_datetime('2022-03-31')
    mask = c['ds'] < threshold_date
    # Split the data and select `ds` and `y` columns.
    df_train = c[mask][['ds', 'y']]
    df_test = c[~ mask][['ds', 'y']]
    
    
    m = Prophet(interval_width=0.95,daily_seasonality=True , seasonality_mode = 'multiplicative'
                ,weekly_seasonality = True, mcmc_samples = 100)
    m.add_seasonality(name='monthly', period=30.5, fourier_order=5)
    
    
    model = m.fit(c)
    future = m.make_future_dataframe(periods=30,freq='D')
    forecast = m.predict(future)
    
    mask2 = forecast['ds'] < threshold_date

    forecast_train = forecast[mask2]
    forecast_test = forecast[~ mask2]

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_true=df_train['y'], y_pred=forecast_train['yhat'],squared = False)



In [ ]:
import seaborn as sns
sns.set_style('darkgrid', {'axes.facecolor': '.9'})
sns.set_palette(palette='deep')
sns_c = sns.color_palette(palette='deep')
%matplotlib inline

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [ ]:
fig, ax = plt.subplots()

ax.fill_between(
    x=forecast['ds'],
    y1=forecast['yhat_lower'],
    y2=forecast['yhat_upper'],
    color=sns_c[2], 
    alpha=0.25,
    label=r'0.95 credible_interval'
)

sns.lineplot(x='ds', y='y', label='y_train', data=df_train, ax=ax)
sns.lineplot(x='ds', y='y', label='y_test', data=df_test, ax=ax)
sns.lineplot(x='ds', y='yhat', label='y_hat', data=forecast, ax=ax)
ax.axvline(threshold_date, color=sns_c[3], linestyle='--', label='train test split')
ax.legend(loc='upper left')
ax.set(title='Dependent Variable', ylabel='');

In [ ]:
#ZOOM

fig, ax = plt.subplots()

ax.fill_between(
    x=forecast_test['ds'],
    y1=forecast_test['yhat_lower'],
    y2=forecast_test['yhat_upper'],
    color=sns_c[2], 
    alpha=0.25,
    label=r'0.95 credible_interval'
)

sns.lineplot(x='ds', y='y', label='y_test', data=df_test, ax=ax)
sns.lineplot(x='ds', y='yhat', label='y_hat', data=forecast_test, ax=ax)
ax.legend(loc='lower right')
ax.set(title='Dependent Variable', ylabel='');

In [ ]:
m.plot(forecast)

In [ ]:
m.plot_components(forecast)

In [ ]:
# from prophet.diagnostics import cross_validation
# from prophet.diagnostics import performance_metrics
# import itertools
# import numpy as np
# import pandas as pd

# param_grid = {  
#     'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
#     'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
# }

# #cutoffs = pd.to_datetime([ '2021-08-15', '2022-02-15'])
# # Generate all combinations of parameters
# all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
# rmses = []  # Store the RMSEs for each params here

# # Use cross validation to evaluate all parameters
# for params in all_params:
#     m = Prophet(**params).fit(c)  # Fit model with given params
#     df_cv = cross_validation(m, horizon='30 days', parallel="processes")
#     df_p = performance_metrics(df_cv, rolling_window=1)
#     rmses.append(df_p['rmse'].values[0])

# # Find the best parameters
# tuning_results = pd.DataFrame(all_params)
# tuning_results['rmse'] = rmses
# print(tuning_results)


# #0.100                     0.01